<a href="https://colab.research.google.com/github/yosshi3/python-season3/blob/master/pytorch/PyTorch_Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chapter2　最尤推定と線形モデル

In [ ]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install tqdm


In [ ]:
import torch
torch.tensor([1,2,3]).to("cuda:0")

リスト2.1　テストデータを生成してパラメータを学習するための変数を準備

In [ ]:
import torch

# 真の係数
w_true = torch.Tensor([1, 2, 3])

# Xのデータの準備。切片を回帰係数に含めるため、
# Xの最初の次元に1を追加しておく
X = torch.cat([torch.ones(100, 1), torch.randn(100, 2)], 1)

# 真の係数と各Xとの内積を行列とベクトルの積でまとめて計算
y = torch.mv(X, w_true) + torch.randn(100) * 0.5

# 勾配降下で最適化するためのパラメータのTensorを
# 乱数で初期化して作成
w = torch.randn(3, requires_grad=True)

# 学習率
gamma = 0.1

リスト2.2　勾配降下法でパラメータを最適化

In [ ]:
# 損失関数のログ
losses = []

# 100回イテレーションを回す
for epoc in range(100):

  # 前回のbackwardメソッドで計算された勾配の値を削除
    w.grad = None
    
    # 線形モデルでyの予測値を計算
    y_pred = torch.mv(X, w)
    
    # MSE lossとwによる微分を計算
    loss = torch.mean((y - y_pred)**2)
    loss.backward()
    
    # 勾配を更新する
    # wをそのまま代入して更新すると異なるTensorになって
    # 計算グラフが破壊されてしまうのでdataだけを更新する
    w.data = w.data - gamma * w.grad.data
    
    # 収束確認のためにlossを記録しておく
    losses.append(loss.item()) 

リスト2.3　matplotlibでプロット

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(losses)

リスト2.4　回帰係数の確認

In [ ]:
w

リスト2.5　線形回帰モデルの構築と最適化の準備

In [ ]:
from torch import nn, optim

# Linear層を作成。今回は切片項は回帰係数に含めるので
# 入力の次元を3とし、bias(切片)をFalseにする
net = nn.Linear(in_features=3, out_features=1, bias=False)

# SGDのオプティマイザーに上で定義したネットワークの
# パラメータを渡して初期化
optimizer = optim.SGD(net.parameters(), lr=0.1)

# MSE lossクラス
loss_fn = nn.MSELoss()

リスト2.6　最適化のイテレーション（繰り返しループ）を回す

In [ ]:
# 損失関数のログ
losses = []

# 100回イテレーションを回す
for epoc in range(100):
  
    # 前回のbackwardメソッドで計算された勾配の値を削除
    optimizer.zero_grad()
    
    # 線形モデルでyの予測値を計算
    y_pred = net(X)
    
    # MSE lossを計算
    # y_predは(n,1)のようなshapeを持っているので(n,)に直す必要がある
    loss = loss_fn(y_pred.view_as(y), y)
    
    # lossのwによる微分を計算
    loss.backward()
    
    # 勾配を更新する
    optimizer.step()
    
    # 収束確認のためにlossを記録しておく
    losses.append(loss.item())

リスト2.7　収束したモデルのパラメータを確認

In [ ]:
list(net.parameters())

リスト2.8　irisのデータセットの準備

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

# irisは(0,1,2)の3クラスの分類問題なのでここでは
# (0,1)の2クラス分のデータだけを使用する
# 本来は訓練用とテスト用に分けるべきだがここでは省略
X = iris.data[:100]
y = iris.target[:100]

# NumPyのndarrayをPyTorchのTensorに変換
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

リスト2.9　モデルの作成

In [ ]:
# irisのデータは4次元
net = nn.Linear(4, 1)

# シグモイド関数を作用させ、2クラス分類の、
# クロスエントロピーを計算する関数
loss_fn = nn.BCEWithLogitsLoss()

# SGD(少し大きめの学習率)
optimizer = optim.SGD(net.parameters(), lr=0.25)

リスト2.10　パラメータ最適化のイテレーションを回す

In [ ]:
# 損失関数のログ
losses = []

# 100回イテレーションを回す
for epoc in range(100):

  # 前回のbackwardメソッドで計算された勾配の値を削除
    optimizer.zero_grad()
    
    # 線形モデルでyの予測値を計算
    y_pred = net(X)
    
    # MSE lossとwによる微分を計算
    loss = loss_fn(y_pred.view_as(y), y)
    loss.backward()
    
    # 勾配を更新する
    optimizer.step()
    
    # 収束確認のためにlossを記録しておく
    losses.append(loss.item())

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(losses)

リスト2.11　モデルの作成

In [ ]:
# 線型結合の結果
h = net(X)

# シグモイド関数を作用させた結果はy=1の確率を表す
prob = nn.functional.sigmoid(h)

# 確率が0.5以上のものをクラス1と予想し、それ以外を0とする
# PyTorchにはBool型がないので対応する型としてByteTensorが出力される
y_pred = prob > 0.5

# 予測結果の確認 (yはFloatTensorなのでByteTensor
# に変換してから比較する）
(y.byte() == y_pred.view_as(y)).sum().item()

リスト2.12　10種類の手書きの数字のデータセットの分類問題

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
X = digits.data
y = digits.target
X = torch.tensor(X, dtype=torch.float32)

# CrossEntropyLoss関数はyとしてint64型のTensorを受け取るので注意
y = torch.tensor(y, dtype=torch.int64)

# 出力は10（クラス数）次元
net = nn.Linear(X.size()[1], 10)

# ソフトマックスクロスエントロピー
loss_fn = nn.CrossEntropyLoss()

# SGD
optimizer = optim.SGD(net.parameters(), lr=0.01)

リスト2.13　学習のイテレーション部分

In [ ]:
# 損失関数のログ
losses = []

# 100回イテレーションを回す
for epoc in range(100):

  # 前回のbackwardメソッドで計算された勾配の値を削除
    optimizer.zero_grad()
    
    # 線形モデルでyの予測値を計算
    y_pred = net(X)
    
    # MSE lossとwによる微分を計算
    loss = loss_fn(y_pred, y)
    loss.backward()
    
    # 勾配を更新する
    optimizer.step()
    
    # 収束確認のためにlossを記録しておく
    losses.append(loss.item())

リスト2.14　正解率

In [ ]:
# torch.maxは集計軸を指定すると最大値の他にその位置も返す
_, y_pred = torch.max(net(X), 1)
# 正解率を計算する
(y_pred == y).sum().item() / len(y)